In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import os

output_folder = "cost_only"
os.makedirs(output_folder, exist_ok=True)

browser = webdriver.Chrome()
browser.get('https://mytcas.com/')
time.sleep(2)

keywords = ['วิศวกรรมคอมพิวเตอร์', 'วิศวกรรมปัญญาประดิษฐ์']

for keyword in keywords:
    print(f'\n🔍 กำลังค้นหา: {keyword}')

    # กลับไปหน้าแรกของ mytcas ทุกครั้ง
    browser.get('https://mytcas.com/')
    time.sleep(2)  # (หรือใช้ WebDriverWait รอ element)

    # รอ input ช่อง search (เปลี่ยน selector ให้ตรงกับของจริง)
    search_box = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.ID, 'search'))
    )
    search_box.clear()
    search_box.send_keys(keyword)
    time.sleep(1)
    search_box.send_keys(Keys.RETURN)

    # รอผลลัพธ์
    try:
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 't-programs'))
        )
    except:
        print("⛔ ไม่พบ ul.t-programs")
        continue

    soup = BeautifulSoup(browser.page_source, 'html.parser')
    ul = soup.find('ul', class_='t-programs')
    links = []

    if ul:
        for li in ul.find_all('li'):
            a = li.find('a', href=True)
            if a:
                links.append(a['href'])

    print(f"🔗 พบลิงก์ทั้งหมด: {len(links)} ลิงก์")

    output_path = os.path.join(output_folder, f"{keyword}.txt")
    with open(output_path, "w", encoding="utf-8") as f:
        for i, link in enumerate(links, 1):
            print(f"  ({i}) เปิดลิงก์: {link}")
            try:
                browser.get(link)
                # รอ <dt> ค่าใช้จ่าย (หรืออะไรก็ได้ใน details)
                WebDriverWait(browser, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, 'dt'))
                )
                detail_soup = BeautifulSoup(browser.page_source, 'html.parser')

                dt_tags = detail_soup.find_all('dt')
                cost_value = "ไม่พบ"

                for dt in dt_tags:
                    if dt.get_text(strip=True) == "ค่าใช้จ่าย":
                        dd = dt.find_next_sibling('dd')
                        if dd:
                            cost_value = dd.get_text(strip=True)
                        break

                f.write(f"{cost_value}\n")
            except Exception as e:
                print(f"    ❗ Error: {e}")
                f.write("ไม่พบ\n")

print("\n✅ เสร็จสิ้น ดึงค่าใช้จ่ายเรียบร้อย")
browser.quit()


🔍 กำลังค้นหา: วิศวกรรมคอมพิวเตอร์
🔗 พบลิงก์ทั้งหมด: 57 ลิงก์
  (1) เปิดลิงก์: https://course.mytcas.com/programs/10010121300501A?
  (2) เปิดลิงก์: https://course.mytcas.com/programs/10010121300599A?
  (3) เปิดลิงก์: https://course.mytcas.com/programs/10020105300501A?
  (4) เปิดลิงก์: https://course.mytcas.com/programs/10020220300501A?
  (5) เปิดลิงก์: https://course.mytcas.com/programs/10020328300502A?
  (6) เปิดลิงก์: https://course.mytcas.com/programs/10020430300501A?
  (7) เปิดลิงก์: https://course.mytcas.com/programs/10030104300501A?
  (8) เปิดลิงก์: https://course.mytcas.com/programs/10040106300501A?
  (9) เปิดลิงก์: https://course.mytcas.com/programs/10040106300502E?
  (10) เปิดลิงก์: https://course.mytcas.com/programs/10050210300501A?
  (11) เปิดลิงก์: https://course.mytcas.com/programs/10050222300501E?
  (12) เปิดลิงก์: https://course.mytcas.com/programs/10060112300501A?
  (13) เปิดลิงก์: https://course.mytcas.com/programs/10090209300501A?
  (14) เปิดลิงก์: https://course.mytc